# Selección base de datos en KAGGLE
 - Dataset: Vehicle dataset


### Importación de librerias necesarias para la limpieza de datos

In [23]:
import pandas as pd
import numpy as np
import scipy as sp
import janitor  

## Descargamos la base de datos

In [24]:
import kagglehub

path = kagglehub.dataset_download("nehalbirla/vehicle-dataset-from-cardekho")

print("Path to dataset files:", path)

c:\Users\Alejo\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\Alejo\.cache\kagglehub\datasets\nehalbirla\vehicle-dataset-from-cardekho\versions\4


## Corroboramos la base de datos

In [25]:
df = pd.read_csv("C:/Users/Alejo/OneDrive/Escritorio/ciencia_de_datos/Limpieza_de_datos/car data.csv")
df2 = pd.read_csv("C:/Users/Alejo/OneDrive/Escritorio/ciencia_de_datos/Limpieza_de_datos/CAR DETAILS FROM CAR DEKHO.csv")
df3 = pd.read_csv("C:/Users/Alejo/OneDrive/Escritorio/ciencia_de_datos/Limpieza_de_datos/Car details v3.csv")
df4 = pd.read_csv("C:/Users/Alejo/OneDrive/Escritorio/ciencia_de_datos/Limpieza_de_datos/car details v4.csv")

In [26]:
# Lista de rutas y nombres de archivos para mayor claridad
archivos = {
    "car_data": "C:/Users/Alejo/OneDrive/Escritorio/ciencia_de_datos/Limpieza_de_datos/car data.csv",
    "car_details_from_cardekho": "C:/Users/Alejo/OneDrive/Escritorio/ciencia_de_datos/Limpieza_de_datos/CAR DETAILS FROM CAR DEKHO.csv",
    "car_details_v3": "C:/Users/Alejo/OneDrive/Escritorio/ciencia_de_datos/Limpieza_de_datos/Car details v3.csv",
    "car_details_v4": "C:/Users/Alejo/OneDrive/Escritorio/ciencia_de_datos/Limpieza_de_datos/car details v4.csv"
}

In [27]:
# Función para realizar la revisión en cada dataset
def revisar_dataset(ruta, nombre):
    print(f"\nRevisión de la base de datos: {nombre}")
    df = pd.read_csv(ruta)
    
    # 1. Valores nulos
    print("\nValores nulos por columna:")
    print(df.isnull().sum())
    
     # 2. Duplicados
    duplicados = df.duplicated().sum()
    print(f"\nNúmero de filas duplicadas: {duplicados}")

    # 3. Tipos de datos
    print("\nTipos de datos por columna:")
    print(df.dtypes)

    # 4. Valores atípicos en columnas numéricas
    print("\nValores atípicos en columnas numéricas:")
    for columna in df.select_dtypes(include=[np.number]).columns:
        Q1 = df[columna].quantile(0.25)
        Q3 = df[columna].quantile(0.75)
        IQR = Q3 - Q1
        outliers = df[(df[columna] < (Q1 - 1.5 * IQR)) | (df[columna] > (Q3 + 1.5 * IQR))]
        num_outliers = outliers.shape[0]
        print(f"- {columna}: {num_outliers} valores atípicos")

    # 5. Limpieza de nombres de columnas con pyjanitor
    print("\nNombres de columnas antes de limpieza:")
    print(df.columns)
    df = df.clean_names()  # Cambia a minúsculas y reemplaza espacios por guiones bajos
    print("\nNombres de columnas después de limpieza (pyjanitor):")
    print(df.columns)

    # 6. Consistencia en columnas categóricas (si existen)
    print("\nValores únicos en columnas categóricas:")
    for columna in df.select_dtypes(include=['object']).columns:
        print(f"- {columna}: {df[columna].unique()}")

    # 7. Estadísticas descriptivas de columnas numéricas
    print("\nEstadísticas descriptivas de columnas numéricas:")
    print(df.describe())

In [19]:
for nombre, ruta in archivos.items():
    revisar_dataset(ruta, nombre)


Revisión de la base de datos: car_data

Valores nulos por columna:
Car_Name         0
Year             0
Selling_Price    0
Present_Price    0
Kms_Driven       0
Fuel_Type        0
Seller_Type      0
Transmission     0
Owner            0
dtype: int64

Número de filas duplicadas: 2

Tipos de datos por columna:
Car_Name          object
Year               int64
Selling_Price    float64
Present_Price    float64
Kms_Driven         int64
Fuel_Type         object
Seller_Type       object
Transmission      object
Owner              int64
dtype: object

Valores atípicos en columnas numéricas:
- Year: 7 valores atípicos
- Selling_Price: 17 valores atípicos
- Present_Price: 14 valores atípicos
- Kms_Driven: 8 valores atípicos
- Owner: 11 valores atípicos

Nombres de columnas antes de limpieza:
Index(['Car_Name', 'Year', 'Selling_Price', 'Present_Price', 'Kms_Driven',
       'Fuel_Type', 'Seller_Type', 'Transmission', 'Owner'],
      dtype='object')

Nombres de columnas después de limpieza (pyjan

## Realizamos la limpieza de los datos

In [28]:
# Función para realizar la limpieza en cada dataset
def limpiar_dataset(ruta, nombre):
    print(f"\nLimpieza de la base de datos: {nombre}")
    df = pd.read_csv(ruta)

    # 1. Eliminar filas duplicadas
    df = df.drop_duplicates()
    print(f"Filas duplicadas eliminadas.")

    # 2. Manejo de valores nulos
    # Para valores numéricos, puedes decidir si rellenas con la mediana o la media
    for columna in df.select_dtypes(include=[np.number]).columns:
        df[columna].fillna(df[columna].median(), inplace=True)  # Usar mediana o media según los datos
    # Para valores categóricos, se pueden rellenar con un valor como "Desconocido" o el más frecuente
    for columna in df.select_dtypes(include=['object']).columns:
        df[columna].fillna("Desconocido", inplace=True)
    print("Valores nulos tratados.")

    # 3. Limpieza de nombres de columnas con pyjanitor
    df = df.clean_names()  # Cambia a minúsculas y reemplaza espacios por guiones bajos
    print("Nombres de columnas limpiados.")

    # 4. Eliminación de valores atípicos (outliers) en columnas numéricas
    for columna in df.select_dtypes(include=[np.number]).columns:
        Q1 = df[columna].quantile(0.25)
        Q3 = df[columna].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df[columna] = np.where(df[columna] < lower_bound, lower_bound,
                               np.where(df[columna] > upper_bound, upper_bound, df[columna]))
    print("Valores atípicos tratados.")

    # 5. Consistencia en valores de columnas categóricas (si existen)
    # Convierte a minúsculas para estandarizar valores y eliminar espacios extra
    for columna in df.select_dtypes(include=['object']).columns:
        df[columna] = df[columna].str.lower().str.strip()
    print("Consistencia en columnas categóricas aplicada.")

    # Guardar el DataFrame limpio
    df.to_csv(f"{nombre}_limpio.csv", index=False)
    print(f"Archivo limpio guardado como {nombre}_limpio.csv")

# Ejecutar la limpieza en cada archivo
for nombre, ruta in archivos.items():
    limpiar_dataset(ruta, nombre)



Limpieza de la base de datos: car_data
Filas duplicadas eliminadas.
Valores nulos tratados.
Nombres de columnas limpiados.
Valores atípicos tratados.
Consistencia en columnas categóricas aplicada.
Archivo limpio guardado como car_data_limpio.csv

Limpieza de la base de datos: car_details_from_cardekho
Filas duplicadas eliminadas.
Valores nulos tratados.
Nombres de columnas limpiados.
Valores atípicos tratados.
Consistencia en columnas categóricas aplicada.
Archivo limpio guardado como car_details_from_cardekho_limpio.csv

Limpieza de la base de datos: car_details_v3
Filas duplicadas eliminadas.
Valores nulos tratados.
Nombres de columnas limpiados.
Valores atípicos tratados.
Consistencia en columnas categóricas aplicada.
Archivo limpio guardado como car_details_v3_limpio.csv

Limpieza de la base de datos: car_details_v4
Filas duplicadas eliminadas.
Valores nulos tratados.
Nombres de columnas limpiados.
Valores atípicos tratados.
Consistencia en columnas categóricas aplicada.
Archivo l

C:\Users\Alejo\AppData\Local\Temp\ipykernel_19160\2857276438.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[columna].fillna(df[columna].median(), inplace=True)  # Usar mediana o media según los datos
C:\Users\Alejo\AppData\Local\Temp\ipykernel_19160\2857276438.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are s

In [29]:
# Rutas de los archivos limpios
archivos_limpios = {
    "car_data_limpio": "car_data_limpio.csv",
    "car_details_from_cardekho_limpio": "car_details_from_cardekho_limpio.csv",
    "car_details_v3_limpio": "car_details_v3_limpio.csv",
    "car_details_v4_limpio": "car_details_v4_limpio.csv"
}

# Función para cargar y observar las bases de datos limpias
def observar_dataset(nombre_archivo):
    print(f"\nObservación de la base de datos: {nombre_archivo}")
    df = pd.read_csv(nombre_archivo)
    
    # Observa las primeras filas
    print("\nPrimeras filas del dataset:")
    print(df.head())

    # Observa estadísticas generales
    print("\nEstadísticas descriptivas del dataset:")
    print(df.describe(include='all'))

    # Observa la información general del dataset (tipos de datos, valores nulos)
    print("\nInformación general del dataset:")
    print(df.info())

    # Conteo de valores nulos por columna (para verificar que se hayan tratado)
    print("\nValores nulos por columna:")
    print(df.isnull().sum())

    # Conteo de valores únicos por columna (para verificar la consistencia en categorías)
    print("\nValores únicos por columna categórica:")
    for col in df.select_dtypes(include=['object']).columns:
        print(f"{col}: {df[col].nunique()} valores únicos")
    
    return df

# Ejecutar la observación en cada archivo limpio
df_car_data_limpio = observar_dataset(archivos_limpios["car_data_limpio"])
df_car_details_from_cardekho_limpio = observar_dataset(archivos_limpios["car_details_from_cardekho_limpio"])
df_car_details_v3_limpio = observar_dataset(archivos_limpios["car_details_v3_limpio"])
df_car_details_v4_limpio = observar_dataset(archivos_limpios["car_details_v4_limpio"])



Observación de la base de datos: car_data_limpio.csv

Primeras filas del dataset:
  car_name    year  selling_price  present_price  kms_driven fuel_type  \
0     ritz  2014.0           3.35           5.59     27000.0    petrol   
1      sx4  2013.0           4.75           9.54     43000.0    diesel   
2     ciaz  2017.0           7.25           9.85      6900.0    petrol   
3  wagon r  2011.0           2.85           4.15      5200.0    petrol   
4    swift  2014.0           4.60           6.87     42450.0    diesel   

  seller_type transmission  owner  
0      dealer       manual    0.0  
1      dealer       manual    0.0  
2      dealer       manual    0.0  
3      dealer       manual    0.0  
4      dealer       manual    0.0  

Estadísticas descriptivas del dataset:
       car_name         year  selling_price  present_price    kms_driven  \
count       299   299.000000     299.000000     299.000000    299.000000   
unique       98          NaN            NaN            NaN      